In [2]:
import os
import zipfile
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet152
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Activation

In [3]:
# اکسترکت تصاویر از فایل ZIP
extraction_path = '/content/drive/MyDrive/Image_Trash'
with zipfile.ZipFile('/content/drive/MyDrive/Image_Trash/trash_images.zip', 'r') as zip_ref:
    zip_ref.extractall(extraction_path)

In [4]:
# مسیر فایل‌های آموزشی
train_data_dir = extraction_path
test_data_dir = '/content/drive/MyDrive/Image_Trash/trash_images'
model_weights_path = '/content/model_weights.h5'

In [5]:
#Augmentation settings for training and test data

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)


In [6]:
#Determining the number of categories and dimensions of images
num_classes = 6
img_width, img_height = 224, 224


In [7]:
#  create model
model = Sequential()

#model.add(Conv2D(32, (3, 3), input_shape=(img_width, img_height, 3)))
model.add(Conv2D(32, (3, 3), input_shape=(500, 500, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(GlobalAveragePooling2D())
model.add(Dense(1024, activation='relu'))
model.add(Dense(num_classes, activation='sigmoid'))
#model.add(Dense(num_classes, activation='softmax'))


In [8]:
#Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [9]:
#Show a summary of the model
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 498, 498, 32)      896       
                                                                 
 activation (Activation)     (None, 498, 498, 32)      0         
                                                                 
 max_pooling2d (MaxPooling2  (None, 249, 249, 32)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 247, 247, 32)      9248      
                                                                 
 activation_1 (Activation)   (None, 247, 247, 32)      0         
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 123, 123, 32)      0         
 g2D)                                                   

In [10]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode='categorical'
)

model.fit(train_generator, epochs=15  , steps_per_epoch=50, validation_steps=30, validation_data=test_generator)



Found 2527 images belonging to 1 classes.
Found 2527 images belonging to 6 classes.
Epoch 1/15
50/50 [==============================] - 157s 3s/step - loss: 128.8182 - accuracy: 0.1007 - val_loss: 76.3576 - val_accuracy: 0.0490
Epoch 2/15
50/50 [==============================] - 173s 3s/step - loss: 4378.5269 - accuracy: 0.1600 - val_loss: 3577.5364 - val_accuracy: 0.2125
Epoch 3/15
50/50 [==============================] - 152s 3s/step - loss: 86652.0938 - accuracy: 0.1601 - val_loss: 53035.5586 - val_accuracy: 0.1604
Epoch 4/15
50/50 [==============================] - 169s 3s/step - loss: 590716.4375 - accuracy: 0.1801 - val_loss: 147522.4688 - val_accuracy: 0.1906
Epoch 5/15
50/50 [==============================] - 148s 3s/step - loss: 3080786.5000 - accuracy: 0.1601 - val_loss: 981058.1250 - val_accuracy: 0.1823
Epoch 6/15
50/50 [==============================] - 169s 3s/step - loss: 8618798.0000 - accuracy: 0.1601 - val_loss: 1677186.5000 - val_accuracy: 0.0521
Epoch 7/15
50/50 [==

In [11]:
#Sequential creation of models for new layers

from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense


model_conv = Sequential()
model_conv.add(Dense(256, activation='relu'))
model_conv.add(Dropout(0.5))
model_conv.add(Dense(1, activation='sigmoid'))



In [12]:
#Determining the hybrid model
combined_model = Model(inputs=model.input, outputs=model_conv(model.output))

In [13]:
#unLocking the lower layers of the pre-trained model for the hybrid model
for layer in model.layers:
    layer.trainable = True


In [14]:
#Compile the hybrid model
combined_model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])

combined_model.fit(train_generator, epochs=15, steps_per_epoch=50, validation_steps=30, validation_data=test_generator)



Epoch 1/15
50/50 [==============================] - 166s 3s/step - loss: 0.2894 - accuracy: 0.9406 - val_loss: 2.2690 - val_accuracy: 0.1667
Epoch 2/15
50/50 [==============================] - 169s 3s/step - loss: 0.0383 - accuracy: 1.0000 - val_loss: 3.4026 - val_accuracy: 0.1667
Epoch 3/15
50/50 [==============================] - 144s 3s/step - loss: 0.0137 - accuracy: 1.0000 - val_loss: 4.0900 - val_accuracy: 0.1667
Epoch 4/15
50/50 [==============================] - 148s 3s/step - loss: 0.0071 - accuracy: 1.0000 - val_loss: 4.5795 - val_accuracy: 0.1667
Epoch 5/15
50/50 [==============================] - 164s 3s/step - loss: 0.0044 - accuracy: 1.0000 - val_loss: 4.9577 - val_accuracy: 0.1667
Epoch 6/15
50/50 [==============================] - 164s 3s/step - loss: 0.0032 - accuracy: 1.0000 - val_loss: 5.2807 - val_accuracy: 0.1667
Epoch 7/15
50/50 [==============================] - 166s 3s/step - loss: 0.0023 - accuracy: 1.0000 - val_loss: 5.5570 - val_accuracy: 0.1667
Epoch 8/15
50

In [15]:
#Show a summary of the hybrid model
combined_model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_input (InputLayer)   [(None, 500, 500, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 498, 498, 32)      896       
                                                                 
 activation (Activation)     (None, 498, 498, 32)      0         
                                                                 
 max_pooling2d (MaxPooling2  (None, 249, 249, 32)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 247, 247, 32)      9248      
                                                                 
 activation_1 (Activation)   (None, 247, 247, 32)      0         
                                                             

In [16]:
#Storage of model weights
combined_model.save(model_weights_path)


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [17]:
#Added: save combined model with H5 format
combined_model.save("combined_models3.h5")


In [21]:
#Loading the hybrid model
from tensorflow.keras.models import load_model
loaded_model = load_model("/content/combined_models3.h5")


In [22]:
#Added: to ensure correct model loading
loaded_model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_input (InputLayer)   [(None, 500, 500, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 498, 498, 32)      896       
                                                                 
 activation (Activation)     (None, 498, 498, 32)      0         
                                                                 
 max_pooling2d (MaxPooling2  (None, 249, 249, 32)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 247, 247, 32)      9248      
                                                                 
 activation_1 (Activation)   (None, 247, 247, 32)      0         
                                                             

In [23]:
# Evaluate the model with classification_report
from sklearn.metrics import classification_report

test_generator.reset()
y_pred = loaded_model.predict(test_generator)
y_pred = y_pred > 0.5
print(classification_report(test_generator.classes, y_pred))

79/79 [==============================] - 55s 690ms/step
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       403
           1       0.20      1.00      0.33       501
           2       0.00      0.00      0.00       410
           3       0.00      0.00      0.00       594
           4       0.00      0.00      0.00       482
           5       0.00      0.00      0.00       137

    accuracy                           0.20      2527
   macro avg       0.03      0.17      0.06      2527
weighted avg       0.04      0.20      0.07      2527



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
